In [4]:
import os

In [5]:
os.listdir()

['.vscode',
 'attendance_backend_test.ipynb',
 'Images',
 'Output_Reports',
 "PDF's"]

### PDF TO IMAGE CONVERSION

In [8]:
from pdf2image import convert_from_path

# 1. Update this path to the actual 'bin' folder in your Poppler download
# Example: If you unzipped it to C:\Program Files\poppler-23.01.0\
poppler_path = r"C:\Users\Jash\AppData\Local\Programs\poppler-25.07.0\Library\bin" 

pdf_path = "PDF's/AOA VKC.pdf"

try:
    # Pass the poppler_path argument to convert_from_path
    images = convert_from_path(pdf_path, poppler_path=poppler_path)

    for i, image in enumerate(images):
        image.save(f'Images/page_{i+1}.jpg', 'JPEG')

    print(f"Successfully converted {len(images)} pages from {pdf_path} to images.")

except Exception as e:
    print(f"An error occurred: {e}")
    print("Please ensure the poppler_path is correct.")

Successfully converted 1 pages from PDF's/AOA VKC.pdf to images.


### OCR

In [5]:
import pytesseract
from PIL import Image
import cv2
import pandas as pd
import numpy as np

In [6]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [12]:
image = Image.open('Images/page_1.jpg')

In [14]:
print(text)

PARSHVANATH'S CHARITABLE TRUST'S

A. P, SHAH INSTITUTE OF TECHNOLOGY
STUDENT ATTENDANCE REPORT (ep
DEPARTMENT OF COMPUTER ENGINEERING. N97]

Academic Year 2025-29 26
ub: OF

a a
a Aether eerie} | le rob \\otad i fert 24 \7 Jes

[me | | yee |' aso! ipa | Lo)
LNO | STUDENTID STUDENT NAME “ —— ol gom a
Saeco — 8p] ote ZN| xX | s=— ———
| Ss fesse —|-—]* ate idee es” ~~ | 7s ce Pi
J tea rasone arama —| | PT RSS8| RNS] Rg | hans | NERS Gos = — | pa NSH! pS
s[e | 2e0s1 [BRANDARTRASATONESH [| Poel 6A eat ee Ce eth le ete nude
c{-.81iof amiss [SIGGARANORAGMARADEO | ee + | ae é ue:

ie]
Ht

J

i]

[82 | _ 24102200 _ |BONGULWAR POOIA MAKARAND
eres | esteaie _| BOPALKAR JATIN JITENDRA
sare
| 86 | 2aioz0s1_ | i ee
|__88 | 24102017 _ [DESHPANDE TANISHQ SACHIN pes i. Ca
aie B Ae
be — ee ees

| 89 | 24102201 _|DIGASKAR TEJAS SANTOSH
[90 | 24102107 [DORLEHARSHSANTOSH |_} 7 ye

2 poe ST VOEANSIE NOS OSE AS tem

i | + | —_

[a1 | 24102065 [DOSHI METI MAYUR
|_92 | 24102043 [DUNGARWAL SIDDHI ASHISH
| 97 |


In [16]:
image = cv2.imread('Images/page_1.jpg')

In [18]:
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [20]:
# Apply binarization (e.g., black and white) to reduce complexity
_, binary_image = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

In [22]:
# 2. Table and Text Extraction (Using a special Tesseract mode)
# Use a Page Segmentation Mode (PSM) configured for a single block of text or a table
custom_config = r'--oem 3 --psm 6' 
extracted_text = pytesseract.image_to_string(binary_image, config=custom_config) # Extracts text

In [23]:
extracted_text

'el\nnea :\nz|* : A EERE LEE\n> 5 z Ale :\n\nTL Tob\nHIME il\nSEKLAL\nHae EATS raeP (EF a\nPer ae i ss oe | fed\naa Yi LN ae Me 18] Gi\nna Da alle\neg | CELE ae : At\n\nREA *\n\nSIS)\n'

In [32]:
def preprocess_image(img_path):
    """Loads image, converts to grayscale, and applies thresholding."""
    # Read the image
    img = cv2.imread(img_path)
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply thresholding (e.g., adaptive thresholding for varied lighting)
    # This separates the text/lines from the background
    binary_img = cv2.adaptiveThreshold(
        gray, 255, 
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
        cv2.THRESH_BINARY_INV, 11, 2
    )
    
    # Use morphological operations (dilation/erosion) to thicken lines/close gaps
    kernel = np.ones((2, 2), np.uint8) # Small kernel to close minor breaks
    dilated_img = cv2.dilate(binary_img, kernel, iterations=1)
    
    return img, dilated_img

In [33]:
def extract_table_data(processed_img):
    """
    Uses pytesseract's detailed output (image_to_data) to get coordinates 
    and text, then structures it into a DataFrame.
    """
    # Use a configuration optimized for a sparse table (PSM 6: Assume a single uniform block of text)
    config = r'--oem 3 --psm 6'

    # Get verbose data, including bounding box coordinates for every detected word/character
    data = pytesseract.image_to_data(processed_img, output_type=pytesseract.Output.DATAFRAME, config=config)
    
    # Filter out empty or low-confidence lines
    data = data.dropna(subset=['text'])
    data = data[data.conf > 30] 
    
    if data.empty:
        print("Error: No text extracted. Check image quality and Tesseract path.")
        return pd.DataFrame()

    # --- Reconstruction Logic (Highly simplified and relies on sorting coordinates) ---
    
    # Sort by 'top' (Y-coordinate) to group words into rows
    # Then sort by 'left' (X-coordinate) within each row
    data = data.sort_values(by=['top', 'left'])

    # Conceptual approach: Group data into virtual cells based on Y-coordinates
    # In a full solution, you would cluster X-coordinates to determine column boundaries.
    
    # For simplicity, we'll try to use the raw text output structured by Tesseract (tesseract HOCR format)
    # The image_to_data output is better for complex tables:
    
    # To reconstruct the grid, you need to define column boundaries.
    # For a dense table like yours, a common approach is to:
    # 1. Identify all unique X-coordinates (or clusters of X-coordinates) to define vertical column lines.
    # 2. Iterate through the data, placing each 'text' into the correct column based on its 'left' coordinate.
    
    # Simplified structure attempt (may result in merged cells/bad alignment):
    # This uses Tesseract's natural grouping into lines (level 4)
    df_raw = data.query('level == 4').groupby('block_num')['text'].apply(lambda x: ' '.join(x)).reset_index()

    # Further complex processing is needed here to split 'text' by columns.
    # Because this requires advanced clustering (Computer Vision + Data Science), 
    # we'll return a simple but potentially messy DataFrame.
    
    return data # Returning the raw data with coordinates for debugging

In [34]:
if __name__ == "__main__":

    IMAGE_PATH = 'Images/page_1.jpg'
    OUTPUT_EXCEL = 'Output_Reports/Raw_OCR_Data.xlsx'
    
    img_color, img_processed = preprocess_image(IMAGE_PATH)
    raw_data_df = extract_table_data(img_processed)
    
    if not raw_data_df.empty:
        
        # --- Custom Table Reconstruction Placeholder ---
        # The reconstruction code is too complex for a template and is HIGHLY dependent on the image.
        # Libraries like img2table (Option 1) handle this complex reconstruction internally.
        
        # For this template, we export the raw coordinate data for analysis
        output_df = raw_data_df[['left', 'top', 'text', 'conf']].copy()
        
        # In a real solution, you would convert this into a clean N x M table (M rows, N columns)
        # Final_Attendance_DF = reconstruct_grid(raw_data_df)
        
        # Export the coordinate data to Excel
        output_df.to_excel(OUTPUT_EXCEL, index=False)
        print(f"\nSUCCESS: Raw OCR data with coordinates saved to {OUTPUT_EXCEL}")
        print("Note: This output needs manual processing to form a clean table.")


SUCCESS: Raw OCR data with coordinates saved to Output_Reports/Raw_OCR_Data.xlsx
Note: This output needs manual processing to form a clean table.


In [ ]:


# --- Configuration ---
# 1. Update this path to your Tesseract executable (MANDATORY on Windows)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' 

# 2. Input/Output Files
IMAGE_PATH = 'page_1.jpg'
OUTPUT_EXCEL = 'attendance_grid_output.xlsx'

# --- 1. Image Processing: Detect Grid Lines ---

def detect_and_clean_grid(img_path):
    """Detects and isolates the horizontal and vertical lines in the image."""
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 1. Thresholding to create a binary image
    _, binary_img = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY_INV) # Threshold may need tuning
    
    # 2. Define Kernels for morphological operations
    # Horizontal kernel (wide and short) to detect horizontal lines
    kernel_h = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1)) 
    # Vertical kernel (tall and thin) to detect vertical lines
    kernel_v = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 40)) 

    # 3. Apply Morphological Operations
    
    # Isolate horizontal lines
    horz_lines = cv2.erode(binary_img, kernel_h, iterations=1)
    horz_lines = cv2.dilate(horz_lines, kernel_h, iterations=1)
    
    # Isolate vertical lines
    vert_lines = cv2.erode(binary_img, kernel_v, iterations=1)
    vert_lines = cv2.dilate(vert_lines, kernel_v, iterations=1)
    
    # 4. Combine the lines
    full_grid = cv2.add(horz_lines, vert_lines)
    
    # 5. Find contours of the full grid (these define the cells)
    contours, _ = cv2.findContours(full_grid, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    return img, contours, horz_lines, vert_lines # Returning lines for later use

# --- 2. Table Reconstruction (Complex: Requires Mapping Lines to Text) ---

def reconstruct_table(img, contours, vert_lines, horz_lines):
    """
    Conceptual function to map OCR text back into the grid defined by lines.
    THIS PART IS HIGHLY COMPLEX and usually requires iterative fine-tuning.
    """
    
    # Find the average X and Y positions of lines to define true column/row splits
    
    # --- Step A: Define Column (X) and Row (Y) Boundaries ---
    
    # 1. Detect X-coordinates of vertical lines to define columns
    vert_projection = np.sum(vert_lines, axis=0) # Sum pixel values vertically
    # Find peaks in the projection to get column separation points
    # (Requires sophisticated peak detection/clustering)
    
    # 2. Detect Y-coordinates of horizontal lines to define rows
    horz_projection = np.sum(horz_lines, axis=1) # Sum pixel values horizontally
    # Find peaks in the projection to get row separation points
    
    # --- Step B: Iterate and OCR Each Cell ---
    
    final_table = []
    
    # This loop is conceptual, as getting the *exact* coordinates for cropping is the challenge
    for r_start, r_end in (conceptual_row_boundaries):
        row_data = []
        for c_start, c_end in (conceptual_column_boundaries):
            
            # 1. Crop the cell area from the original image
            cell_img = img[r_start:r_end, c_start:c_end]
            
            # 2. OCR the cropped cell
            # Use PSM 10 (Single Character) or PSM 8 (Single Word) for attendance marks
            cell_text = pytesseract.image_to_string(cell_img, config='--psm 10').strip()
            
            # 3. Apply your custom cleaning/mapping function (P/A/Dot -> Present/Absent/Unclear)
            # cell_text = clean_attendance_marks(cell_text) # Your previous function
            
            row_data.append(cell_text)
        final_table.append(row_data)

    # Convert the final list of lists into a Pandas DataFrame
    df = pd.DataFrame(final_table)
    return df

    


# --- 3. Main Execution and Export ---

if __name__ == "__main__":
    
    try:
        img_color, cell_contours, horz_lines, vert_lines = detect_and_clean_grid(IMAGE_PATH)
        
        # Display the detected lines (Optional, for debugging)
        # cv2.imwrite('detected_grid.jpg', cv2.add(horz_lines, vert_lines))
        # print("Saved 'detected_grid.jpg' for visual debugging.")
        
        final_df = reconstruct_table(img_color, cell_contours, vert_lines, horz_lines)

        # if not final_df.empty:
        #     final_df.to_excel(OUTPUT_EXCEL, index=False)
        #     print(f"\nSUCCESS: Data saved to {OUTPUT_EXCEL}")

    except Exception as e:
        print(f"\nAn error occurred during execution: {e}")
        print("Ensure Tesseract and all libraries are installed.")

In [51]:

# --- 1. Image Processing ---

def deskew_and_preprocess(img_path):
    """
    Attempts to detect and correct image skew using Tesseract OSD, 
    then converts to a binary image.
    """
    try:
        # Open the image with PIL
        img_pil = Image.open(img_path)
        
        # Get Orientation from Tesseract (PSM 0 is for OSD only)
        osd_config = r'--psm 0'
        osd_data = pytesseract.image_to_osd(img_pil, config=osd_config)
        
        # Simple parsing to get the angle
        angle_match = next((line for line in osd_data.split('\n') if line.startswith('Rotate:')), None)
        rotation_angle = int(angle_match.split(': ')[1]) if angle_match else 0

        print(f"Detected rotation angle: {rotation_angle}°")
        
        if rotation_angle != 0:
            print(f"Deskewing: Detected rotation angle: {rotation_angle}°")
            # Rotate and expand to ensure no content is clipped
            rotated_img_pil = img_pil.rotate(rotation_angle, expand=True)
        else:
            rotated_img_pil = img_pil

        # Convert PIL Image back to OpenCV format
        img_np = np.array(rotated_img_pil.convert('RGB'))
        img_color = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
        
        # Convert to grayscale and apply thresholding
        gray = cv2.cvtColor(img_color, cv2.COLOR_BGR2GRAY)
        # Use a high threshold for better contrast on marks, adjust 180 as needed
        _, binary_img = cv2.threshold(gray, 180, 255, cv2.THRESH_BINARY)
        
        return binary_img

    except pytesseract.TesseractNotFoundError:
        print("Tesseract not found. Check tesseract_cmd path!")
        return None
    except Exception as e:
        print(f"Error during deskewing/preprocessing: {e}")
        return None




In [48]:
def extract_table_data(processed_img):
    """
    Uses pytesseract's detailed output (image_to_data) to get coordinates 
    and text, then structures it into a DataFrame.
    """
    # Use PSM 6: Assume a single uniform block of text.
    config = r'--oem 3 --psm 6' 

    # Get verbose data: 'left', 'top', 'height', 'width', 'text', 'conf', etc.
    data = pytesseract.image_to_data(processed_img, output_type=pytesseract.Output.DATAFRAME, config=config)
    
    # Filter out empty text and low-confidence results
    data = data.dropna(subset=['text'])
    data = data[data['text'].str.strip() != '']
    data = data[data['conf'] > 30] 
    
    if data.empty:
        return pd.DataFrame()

    # --- Reconstruction Logic using Spatial Coordinates (top, left) ---
    
    # 1. Sort spatially
    data = data.sort_values(by=['top', 'left'])

    # 2. Custom Row Grouping (Tolerance for the same row)
    row_tolerance = 20 # Adjust this value based on your cell height
    data['row_id'] = (data['top'].diff() > row_tolerance).cumsum()

    # 3. Custom Column Grouping (Clustering X-coordinates)
    left_positions = data['left'].unique()
    left_positions.sort()

    column_clusters = []
    current_cluster = []
    
    for x in left_positions:
        if not current_cluster:
            current_cluster.append(x)
        elif x - current_cluster[-1] < 50: # Tolerance of 50 pixels for the same column
            current_cluster.append(x)
        else:
            column_clusters.append(int(np.mean(current_cluster)))
            current_cluster = [x]
    if current_cluster:
        column_clusters.append(int(np.mean(current_cluster)))

    def get_column_id(left_coord):
        return np.argmin(np.abs(np.array(column_clusters) - left_coord))

    data['col_id'] = data['left'].apply(get_column_id)

    # 4. Pivot the data to create the final DataFrame
    # Note: Using max() or first() might merge words incorrectly; this is the biggest point of failure.
    #final_df = data.pivot(index='row_id', columns='col_id', values='text').fillna('').apply(lambda x: ' '.join(x), axis=1)
    
    # After pivoting and joining, the column names are still messy numbers.
    # We must manually clean the data.
    
    # Due to the complexity of the column structure in your image (Roll No, Name, then Dates),
    # this resulting DataFrame requires one final cleanup pass.
    
    #return final_df.to_frame() # Return as a DataFrame
    return data

In [13]:
# --- Configuration ---
# 1. Update this path to your Tesseract executable (MANDATORY on Windows)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' 

# 2. Input/Output Files
IMAGE_PATH = 'Images/page_1.jpg'
OUTPUT_EXCEL = 'attendance_grid_output.xlsx'

In [52]:
img_processed = deskew_and_preprocess(IMAGE_PATH)

Detected rotation angle: 0°


In [49]:
if img_processed is not None:
    # Step 2: Extract Data and Structure
    final_df = extract_table_data(img_processed)

In [50]:
final_df

,level,page_num,block_num,par_num,line_num,word_num,left,top,width,height,conf,text,row_id,col_id
9,5,1,1,1,2,4,1400,344,2,2,41.559139,|,0,12
11,5,1,1,1,3,1,175,435,24,66,37.183258,el,1,0
12,5,1,1,1,3,2,215,435,28,66,52.886387,&),1,0
15,5,1,1,1,3,5,336,435,22,66,36.425919,S|,1,2
17,5,1,1,1,3,7,413,448,13,20,36.664349,§,1,3
23,5,1,1,1,5,1,164,783,500,112,36.809917,TTT,2,0
30,5,1,1,1,6,5,1102,786,151,116,40.074837,eee,2,8
33,5,1,1,1,7,2,1350,883,22,61,33.002319,PR,3,11
26,5,1,1,1,6,1,128,913,162,24,40.332829,AH,4,0
37,5,1,1,1,8,3,1287,1537,82,31,32.156410,Ga,5,10


In [27]:
import OCRToTableTool as ottt
import TableExtractor as te
import TableLinesRemover as tlr
import cv2
import os

In [28]:

os.listdir("Images")

['page_1.jpg']

In [29]:
path_to_image = "Images/page_1.jpg"
table_extractor = te.TableExtractor(path_to_image)
perspective_corrected_image = table_extractor.execute()

In [19]:
lines_remover = tlr.TableLinesRemover(perspective_corrected_image)
image_without_lines = lines_remover.execute()

In [20]:
ocr_tool = ottt.OcrToTableTool(image_without_lines, perspective_corrected_image)
ocr_tool.execute()

cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
cv2.imshow("perspective_corrected_image", perspective_corrected_image)

: 